# Analysis and Data Visualization
## Marketing Leads data
Olist provides us a raw file that includes:
- All leads including those that did not become approved sellers (by `mql_id`)
- The "first contact" date of each approved seller
- The landing page for each approved seller
- The channel of origin for each approved seller

## Closed Leads data
Olist provides us a raw file that includes
- Approved sellers (by `mql_id`)
- Approved sellers (new `seller_id`, `sdr_id`, and `sr_id` fields)
- The datetime the lead became an approved seller
- The approved seller's business segment
- The approved seller's type (`online_medium`, `online_big`, `industry`, `offline`, etc.)
- The approved seller's behaviour profile
- The approved seller's incorporation status
- The approved seller's GTIN status
- The approved seller's average stock
- The approved seller's business type
- The approved seller's declared product catalog size
- The approved seller's declared monthly revenue

## Questions to answer:
- Does the origin align with marketing spend?
- Is there any discernable seasonality to first contact dates?
- Can Olist focus on one particular channel to maximize the likelihood of convering a lead to an approved seller?

In [2]:
import polars as pl
from lets_plot import *
LetsPlot.setup_html()

In [40]:
mql_csv_filepath = 'C:\\github\\olist\\olist_marketing_qualified_leads_dataset.csv'
mql_df = pl.read_csv(mql_csv_filepath)
mql_df.head(5)

mql_id,first_contact_date,landing_page_id,origin
str,str,str,str
"""dac32acd4db4c2…","""2018-02-01""","""88740e65d5d6b0…","""social"""
"""8c18d1de7f67e6…","""2017-10-20""","""007f9098284a86…","""paid_search"""
"""b4bc852d233dfe…","""2018-03-22""","""a7982125ff7aa3…","""organic_search…"
"""6be030b81c7597…","""2018-01-22""","""d45d558f0daeec…","""email"""
"""5420aad7fec354…","""2018-02-21""","""b48ec5f3b04e90…","""organic_search…"


# Does the origin align with marketing spend?
- First we can aggregate the count of leads by origin
- Then we can convert that count into percentages
- **From those calculations and visualizations we can work with the business to ensure our marketing spend is working as expected**

In [52]:
ggplot(mql_df, aes(x='origin')) + \
    geom_bar(aes(fill='origin'), alpha=1.0)

In [59]:
# Use built-in count, proportion, and total capabilities of lets-plot
tooltip_content = layer_tooltips().line('percent|@{..proppct..} (@{..count..})')\
                                  .line('total|@{..sum..}')
ggplot(mql_df) + \
    geom_pie(aes(fill='origin'),
             size=25,
             hole=0.3,
             labels=layer_labels().line('@origin').size(9),
             tooltips=tooltip_content) + \
    ggsize(600, 400) + \
    theme_void()

### ANSWER: Does the origin align with marketing spend?
- Display at 1.5% seems low, worth investigating that budget vs. other budgets
- Referral likely has an extremely high ROI (word of mouth works), talk with acquisition team about rewards there
- Email is also likely to have an extremely high ROI (they trusted you with their inbox), talk with acquisition team about those campaigns

### Anything else?
- Work with engineering / tech to get a handle on unknown, other, and n/a categories

# Is there any discernable seasonality to first contact dates?
- Make sure polars recognizes dates
- Examine time series data overall (day of week for example) and by origin

In [81]:
mql_date_aware_df = (
    mql_df
    .with_columns(
        pl.col('first_contact_date').str.to_datetime('%Y-%m-%d').alias('dt_first_contact_date')
    )
    .with_columns(
        pl.col('dt_first_contact_date').dt.weekday().alias('dt_first_contact_weekday_numerical'),
        pl.col('dt_first_contact_date').dt.to_string('%A').alias('dt_first_contact_weekday')
    )
    .sort(['dt_first_contact_weekday_numerical', 'dt_first_contact_date'])
)
mql_date_aware_df.head(5)

mql_id,first_contact_date,landing_page_id,origin,dt_first_contact_date,dt_first_contact_weekday_numerical,dt_first_contact_weekday
str,str,str,str,datetime[μs],i8,str
"""c0a4b0fa886250…","""2017-07-03""","""63a22f10d1d63d…","""email""",2017-07-03 00:00:00,1,"""Monday"""
"""b34757e1761fdb…","""2017-07-10""","""c2f7f172d3275d…","""social""",2017-07-10 00:00:00,1,"""Monday"""
"""a809dc00968793…","""2017-07-10""","""c79bafc4a099ff…","""paid_search""",2017-07-10 00:00:00,1,"""Monday"""
"""54aba384bebf7d…","""2017-07-10""","""acd502dc52a068…","""organic_search…",2017-07-10 00:00:00,1,"""Monday"""
"""a2d436321f525b…","""2017-07-10""","""a6ad0162dbe118…","""display""",2017-07-10 00:00:00,1,"""Monday"""


In [82]:
ggplot(mql_date_aware_df.sort('dt_first_contact_weekday'), aes(x='dt_first_contact_weekday')) + \
    geom_bar(aes(fill='dt_first_contact_weekday'), alpha=1.0)

In [83]:
ggplot(mql_date_aware_df, aes(x='dt_first_contact_date')) + \
    geom_line(stat='count', size=1) + \
    scale_x_datetime(format='%b %Y')

In [95]:
ggplot(mql_date_aware_df, aes(x='dt_first_contact_date')) + \
    geom_bar(aes(group='origin', fill='origin', color='origin')) + \
    scale_x_datetime(format='%b %Y')

### ANSWER: Is there any discernable seasonality to first contact dates?
- Friday through Sunday are low, outreach is involved so low Olist sales staffing is probably the cause, good use of resources
- No annual seasonality can be seen (yet), obviously acquisitions ramped up in 2018 but both periods are visually flat

### Anything else?
- Speak with acquisition team about holiday periods in Brazil, very low numbers across all origins in December may be explained by time off

# Can Olist focus on one particular channel to maximize the likelihood of convering a lead to an approved seller?
- Join the approved sellers data with the leads data
- Identify which origin has the best conversion rate
- Find areas to optimize

In [96]:
as_csv_filepath = 'C:\\github\\olist\\olist_closed_deals_dataset.csv'
as_df = pl.read_csv(as_csv_filepath)
as_df.head(5)

mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
str,str,str,str,str,str,str,str,bool,bool,str,str,f64,f64
"""5420aad7fec354…","""2c43fb513632d2…","""a8387c01a09e99…","""4ef15afb4b2723…","""2018-02-26 19:…","""pet""","""online_medium""","""cat""",null,null,null,"""reseller""",null,0.0
"""a555fb36b93681…","""bbb7d7893a4506…","""09285259593c61…","""d3d1e91a157ea7…","""2018-05-08 20:…","""car_accessorie…","""industry""","""eagle""",null,null,null,"""reseller""",null,0.0
"""327174d3648a2d…","""612170e34b9700…","""b90f87164b5f8c…","""6565aa9ce3178a…","""2018-06-05 17:…","""home_appliance…","""online_big""","""cat""",null,null,null,"""reseller""",null,0.0
"""f5fee8f7da74f4…","""21e1781e36faf9…","""56bf83c4bb3576…","""d3d1e91a157ea7…","""2018-01-17 13:…","""food_drink""","""online_small""",null,null,null,null,"""reseller""",null,0.0
"""ffe640179b554e…","""ed8cb7b190ceb6…","""4b339f9567d060…","""d3d1e91a157ea7…","""2018-07-03 20:…","""home_appliance…","""industry""","""wolf""",null,null,null,"""manufacturer""",null,0.0


In [134]:
full_df = mql_date_aware_df.join(as_df, on='mql_id', how='left')
full_df.tail(5)

mql_id,first_contact_date,landing_page_id,origin,dt_first_contact_date,dt_first_contact_weekday_numerical,dt_first_contact_weekday,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
str,str,str,str,datetime[μs],i8,str,str,str,str,str,str,str,str,bool,bool,str,str,f64,f64
"""79c57bf41c3448…","""2018-05-27""","""f017be4dbf8624…","""paid_search""",2018-05-27 00:00:00,7,"""Sunday""","""11742f81550ac0…","""370c9f455f93a9…","""de63de0d10a601…","""2018-05-31 21:…","""home_appliance…","""online_medium""","""wolf""",null,null,null,"""reseller""",null,0.0
"""48a52d5cf371cf…","""2018-05-27""","""22c29808c4f815…","""direct_traffic…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""6f25a1fecb5405…","""2018-05-27""","""22c29808c4f815…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""702c7c3c2936d1…","""2018-05-27""","""b76ef37428e679…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null
"""77fec91772e29a…","""2018-05-27""","""22c29808c4f815…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null


In [135]:
full_with_approved_seller_flag_df = (
    full_df
    .with_columns(
        pl.col('seller_id').is_not_null().alias('approved_seller_flag')
    )
)
full_with_approved_seller_flag_df.tail(5)

mql_id,first_contact_date,landing_page_id,origin,dt_first_contact_date,dt_first_contact_weekday_numerical,dt_first_contact_weekday,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue,approved_seller_flag
str,str,str,str,datetime[μs],i8,str,str,str,str,str,str,str,str,bool,bool,str,str,f64,f64,bool
"""79c57bf41c3448…","""2018-05-27""","""f017be4dbf8624…","""paid_search""",2018-05-27 00:00:00,7,"""Sunday""","""11742f81550ac0…","""370c9f455f93a9…","""de63de0d10a601…","""2018-05-31 21:…","""home_appliance…","""online_medium""","""wolf""",null,null,null,"""reseller""",null,0.0,true
"""48a52d5cf371cf…","""2018-05-27""","""22c29808c4f815…","""direct_traffic…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""6f25a1fecb5405…","""2018-05-27""","""22c29808c4f815…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""702c7c3c2936d1…","""2018-05-27""","""b76ef37428e679…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null,false
"""77fec91772e29a…","""2018-05-27""","""22c29808c4f815…","""organic_search…",2018-05-27 00:00:00,7,"""Sunday""",null,null,null,null,null,null,null,null,null,null,null,null,null,false


In [180]:
# Isolate all the unique origin values
origins = set(full_with_approved_seller_flag_df['origin'])
origins.remove(None)

# Prepare some convenience structures for later graphing
leads_only = []
approved_sellers_only = []
conversion_rates = []
leads_by_origin = {}
approved_sellers_by_origin = {}

# Loop through the unique origin values and collect origin-specific observations
for origin in origins:
    leads_by_origin[origin] = (
        full_with_approved_seller_flag_df
        .filter(pl.col('origin') == origin)
        .count()['mql_id'][0]
    )
    leads_only.append(leads_by_origin[origin])
    
    approved_sellers_by_origin[origin] = (
        full_with_approved_seller_flag_df
        .filter((pl.col('approved_seller_flag'))
                & (pl.col('origin') == origin))
        .count()['mql_id'][0]
    )
    approved_sellers_only.append(approved_sellers_by_origin[origin])
    
    conversion_rates.append(approved_sellers_by_origin[origin] \
                            / leads_by_origin[origin] * 100)

# Prepare polars-friendly dictionaries then dataframes
leads_for_polars = {'origin': list(origins),
                    'leads': leads_only}
leads_df = pl.from_dict(leads_for_polars)

approved_sellers_for_polars = {'origin': list(origins),
                               'approved_sellers': approved_sellers_only}
approved_sellers_df = pl.from_dict(approved_sellers_for_polars)

conversion_rates_for_polars = {'origin': list(origins),
                               'conversion_rate': conversion_rates}
conversion_rate_df = pl.from_dict(conversion_rates_for_polars)

In [181]:
ggplot(approved_sellers_df) + \
    geom_bar(aes(x='origin', y='approved_sellers', group='origin',
                 color='origin', fill='origin'), stat='identity')

In [182]:
ggplot(conversion_rate_df) + \
    geom_bar(aes(x='origin', y='conversion_rate', group='origin',
                 color='origin', fill='origin'), stat='identity')

### ANSWER: Can Olist focus on one particular channel to maximize the likelihood of convering a lead to an approved seller?
- **YES!**
- Paid Search has the 2nd highest conversion rate (12.3%) and the 2nd highest quantity of approved sellers (195), lever up Paid Search because it's working well **(remember that this is not golf, the market rewards exceptional strength, not well-rounded averages)**
- Display has the worst categorized quantity of approved sellers and a modest conversion rate, **take action to improve or drop Display**
- Email has the 2nd worst categorized quantity of approved sellers and a low conversion rate, **take action to improve or drop Email**

### Anything else?
- Work with engineering to get an idea about all those `unknown` conversions (3rd largest group by quantity and the best converter of the bunch)